In [ ]:
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator
import conf_solo12_trot as conf

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

In [ ]:
# # display whole-body motion in meshcat viewer
# from wholebody_croccodyl_solver import WholeBodyDDPSolver
# from wholebody_croccodyl_model import WholeBodyModel
# import pinocchio as pin
# import sys

# wbd_model = WholeBodyModel(conf)
# ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=True, WARM_START=True)
# N_inner = int(conf.dt/conf.dt_ctrl)

# sol = ddp_planner.run_OL_MPC(None, None)

# if conf.WITH_MESHCAT_DISPLAY:
#     viz = pin.visualize.MeshcatVisualizer(
#     robot.model, robot.collision_model, robot.visual_model)
#     try:
#         viz.initViewer(open=True)
#     except ImportError as err:
#         print(
#             "Error while initializing the viewer. Make sure you installed Python meshcat"
#         )
#         print(err)
#         sys.exit(0)
#     viz.loadViewerModel()
#     for sol_k in sol:
#         q_des, qdot_des, tau_ff = ddp_planner.interpolate_one_step(
#         q=sol_k['jointPos'][0], q_next=sol_k['jointPos'][1],
#         qdot=sol_k['jointVel'][0],qdot_next=sol_k['jointVel'][1],
#         tau=sol_k['jointTorques'][0], tau_next=sol_k['jointTorques'][1]
#                 )
#         for ctrl_time_idx in range(N_inner):
#             viz.display(q_des[ctrl_time_idx])

In [ ]:
# simulate motion in pybullet
import numpy as np
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import pinocchio as pin
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=True, WARM_START=True)
running_models = ddp_planner.whole_body_model.running_models
N_traj, N_mpc = ddp_planner.N_traj, ddp_planner.N_mpc
Kp = 2*np.eye(12)
Kd = 0.01*np.eye(12)
N_inner = int(ddp_planner.dt/ddp_planner.dt_ctrl)
for traj_time_idx in range(N_traj):
    print('\n'+'=' * 50)
    print('MPC Iteration ' + str(traj_time_idx))
    print('-' * 50)
    # update models
    current_running_models = running_models[traj_time_idx:traj_time_idx+N_mpc]
    current_terminal_model = current_running_models[-1]
    ddp_planner.update_ocp(traj_time_idx)
    if traj_time_idx == 0:
        if ddp_planner.WARM_START:
            ddp_planner.solver.solve(ddp_planner.x_init, ddp_planner.u_init)
        else:
            ddp_planner.solve()
    else:
        ddp_planner.solve(xs, us)
    sol_k = ddp_planner.get_solution_trajectories()
    # scaling DDP gains (what is the proper way of doing this?)
    gains = sol_k['gains'][0]*(1/N_inner)
    q_des, qdot_des, tau_ff = ddp_planner.interpolate_one_step(
                    q=sol_k['jointPos'][0], q_next=sol_k['jointPos'][1],
                qdot=sol_k['jointVel'][0],qdot_next=sol_k['jointVel'][1],
                tau=sol_k['jointTorques'][0], tau_next=sol_k['jointTorques'][1]
                    )
    for ctrl_time_idx in range(N_inner):
        Kp = gains[:, 6:18]
        Kd = gains[:, 24:]
        q_k, dq_k = simulator.robot.get_state()
        tau_k = tau_ff[ctrl_time_idx] + \
                  Kp @ (q_des[ctrl_time_idx, 7:] - q_k[7:]) + \
                  Kd @ (q_des[ctrl_time_idx, 7:] - dq_k[6:])
        simulator.robot.send_joint_command(tau_k)
        # step simulation 
        simulator.env.step(sleep=True)     
    q_k, dq_k = simulator.robot.get_state()
    # warm-start ocp from the previous solution
    x = [ddp_planner.solver.xs[i] for i in range(len(ddp_planner.solver.xs))]
    u = [ddp_planner.solver.us[i] for i in range(len(ddp_planner.solver.us))]
    us = u[1:] + [u[-1]]    
    xs = x[1:] + [x[-1]]
    # OL-MPC (initial state = predicted state)
    # x0 = xs[0]
    # CL-MPC (initial state = measured state)
    x0 = np.concatenate([q_k, dq_k])
    ddp_planner.x0 = x0


: 